In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import dbcp
import pandas as pd
import numpy as np
from pathlib import Path

/app/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(
/app/.local/lib/python3.10/site-packages/pudl/analysis/spatial.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/

In [4]:
path_new = Path("/app/data/raw/2023.05.30 Opposition to Renewable Energy Facilities - FINAL.docx")
assert path_new.exists()

In [5]:
new = dbcp.extract.local_opposition.ColumbiaDocxParser()
new.load_docx(path_new)

In [6]:
extracted_new = new.extract()

In [7]:
extracted_new['state_notes'].shape, extracted_new['state_policy'].shape

((16, 2), (10, 2))

In [8]:
with pd.option_context("display.max_colwidth", None):
    display(extracted_new['state_notes'].head(3))


,state,notes
0,California,"Note on state preemption of local restrictions: On June 30, 2022, the State of California adopted legislation that allows the California Energy Commission to bypass local laws when permitting large-scale renewable energy projects. Under the new law, the commission has jurisdiction to issue a certificate for any: (a) photovoltaic solar facility, on-shore wind facility, or thermal energy facility not powered by fossil fuels or nuclear fuels, with a generating capacity of at least 50 MW; (b) energy storage system with a storage capacity of least 200 MW hours (MWh); (c) electric transmission line from any such generating or storage facility to an interconnected transmission system; and (d) facility that manufactures, produces, or assembles wind, solar, or storage systems with a capital investment of at least $250,000,000 over a period of 5 years. The law provides that the issuance of a certificate shall “be in lieu of any permit, certificate, or similar document required by any state, local, or regional agency,” and “supersede any applicable statute, ordinance, or regulation of any state, local, or regional agency.”"
1,Connecticut,"Note on state preemption of local restrictions: The Connecticut Siting Council has exclusive jurisdiction over renewable energy facilities, excluding certain emergency facilities with a capacity of 1 MW or less. While state law allows local governments to “regulate and restrict the location of a proposed location of a facility,” the Siting Council can approve projects that violate local zoning restrictions. The Appellate Court of Connecticut has held that the Connecticut Siting Council “is empowered to review decisions from zoning commissions on a de novo basis, applying concerns that transcend those involved in local zoning decisions, and that review may . . . result in the approval of a particular site although the facility failed to meet the requirements of local zoning regulations.”"
2,Florida,"Note on state preemption of local restrictions: In Florida, a siting board comprised of the governor and the governor’s cabinet has the authority to set aside local restrictions on a case-by-case basis. If the siting board determines that a proposed facility “does not conform with existing land use plans and zoning ordinances,” it may “authorize a variance or other necessary approval to the adopted land use plan and zoning ordinances required to render the proposed site or associated facility consistent with local land use plans and zoning ordinances.” Further, in 2021, Florida enacted legislation preventing local governments from restricting solar energy facilities on agricultural land. The law provides that “[a] solar facility shall be a permitted use in all agricultural land use categories in a local government comprehensive plan and all agricultural zoning districts within an unincorporated area.” The law further provides that, although counties may adopt ordinances specifying buffer and landscaping requirements, any such requirements “may not exceed the requirements for similar uses involving the construction of other facilities that are permitted uses in agricultural land use categories and zoning districts.”"


In [9]:
extracted_new['state_policy'].loc[extracted_new['state_policy'].duplicated(subset=['state'], keep=False), :]

,state,policy


In [10]:
extracted_new['state_policy']

,state,policy
0,Connecticut,"In 2017, the Legislature enacted Public Act No..."
1,Iowa,No restrictive state laws or policies were fou...
2,Kansas,"In 2004, Kansas Governor Kathleen Sebelius ins..."
3,Kentucky,"Under House Bill 291, adopted in 2014, wind tu..."
4,Maine,Governor LePage signed an executive order in J...
5,Minnesota,A Minnesota administrative rule provides that ...
6,New York,"On May 3, 2023, the state adopted a law direct..."
7,Ohio,"Ohio Senate Bill 52 of 2021 (“S.B. 52”), which..."
8,Oklahoma,"Since August 21, 2015, Oklahoma has required t..."
9,Oregon,"On May 23, 2019, the Oregon Department of Land..."


In [17]:
extracted_new['local_ordinance'].shape

(228, 3)

In [19]:
ordn = extracted_new['local_ordinance'].copy()

In [20]:
ordn.head()

,state,locality,ordinance_text
0,Alabama,Baldwin County,Wind energy systems with a capacity of 50 kW o...
1,Alabama,Cherokee County,"In 2014, the Alabama state legislature enacted..."
2,Alabama,DeKalb County,"In DeKalb County, wind turbines must be set ba..."
3,Alabama,Etowah County,"In 2014, the Alabama state legislature enacted..."
4,California,Los Angeles County,The Los Angeles County Code prohibits “[u]tili...


In [21]:
ordn.duplicated(subset=['state', 'locality'], keep=False).sum()

0

In [22]:
ordn['locality'].str.contains('(', regex=False).sum()

91

In [23]:
ordn['locality'].str.lower().str.contains('county', regex=False).sum()

220

In [24]:
ordn.loc[~ordn['locality'].str.lower().str.contains('county', regex=False), :]

,state,locality,ordinance_text
62,Louisiana,St. James Parish,"On August 17, 2022, the St. James Parish Counc..."
63,Louisiana,Jefferson Davis Parish,"Under a January 2020 ordinance, no wind turbin..."
179,Oklahoma,City of Owasso (Rogers and Tulsa Counties),A 2019 ordinance prohibits wind turbines over ...
195,Texas,City of Burleson (Tarrant and Johnson Counties),The City of Burleson stipulates that turbines ...
197,Texas,"City of Garland (Collin, Dallas, and Rockwall ...",An ordinance adopted May 2015 provides that wi...
198,Texas,"City of Grand Prairie (Dallas, Tarrant, and El...",The only wind energy systems allowed in the Ci...
203,Texas,City of Ovilla (Dallas and Ellis Counties),The city zoning ordinance provides that noise ...
204,Texas,City of Waller (Waller and Harris Counties),The City of Waller allows small wind energy sy...


In [28]:
transformed = dbcp.transform.local_opposition.transform(extracted_new)

2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:203 Assigned state FIPS codes for 100.00% of records.
2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:219 Assigned county FIPS codes for 0.00% of records.
2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:203 Assigned state FIPS codes for 100.00% of records.
2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:219 Assigned county FIPS codes for 0.00% of records.
2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:203 Assigned state FIPS codes for 100.00% of records.
2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:219 Assigned county FIPS codes for 60.09% of records.
2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:203 Assigned state FIPS codes for 100.00% of records.
2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:219 Assigned county FIPS codes for 100.00% of records.
2023-06-21 01:48:12 [    INFO] catalystcoop.pudl.helpers:203 Assigned state FIPS codes for 100.00% of records.
20

In [25]:
trans_ordn = transformed['local_ordinance'].copy()

In [26]:
# geocoding takes care of the multi-county cities with no special treatment
col_subset = [
        "raw_state_name",
        "raw_locality_name",
        "county_id_fips",
        "geocoded_locality_name",
        "geocoded_locality_type",
        "geocoded_containing_county",
    ]
trans_ordn.loc[
    ~trans_ordn["raw_locality_name"].str.lower().str.contains("county", regex=False),
    col_subset,
]


,raw_state_name,raw_locality_name,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
62,Louisiana,St. James Parish,22093,St. James Parish,county,St. James Parish
63,Louisiana,Jefferson Davis Parish,22053,Jefferson Davis Parish,county,Jefferson Davis Parish
179,Oklahoma,City of Owasso (Rogers and Tulsa Counties),40143,Owasso,city,Tulsa County
195,Texas,City of Burleson (Tarrant and Johnson Counties),48251,Burleson,city,Johnson County
197,Texas,"City of Garland (Collin, Dallas, and Rockwall ...",48397,Rockwall,city,Rockwall County
198,Texas,"City of Grand Prairie (Dallas, Tarrant, and El...",48113,Grand Prairie,city,Dallas County
203,Texas,City of Ovilla (Dallas and Ellis Counties),48139,Ovilla,city,Ellis County
204,Texas,City of Waller (Waller and Harris Counties),48473,Waller,city,Waller County


In [27]:
geo = [col for col in col_subset if 'geo' in col]
geo

['geocoded_locality_name',
 'geocoded_locality_type',
 'geocoded_containing_county']

I altered the transform code to remove county names prior to geocoding and compared the results below.

In [48]:
not_equal = transformed['local_ordinance'][geo].ne(trans_ordn[geo]).any(axis=1)

In [50]:
trans_ordn.loc[not_equal, col_subset].join(transformed['local_ordinance'].loc[not_equal, geo], rsuffix='_new')

,raw_state_name,raw_locality_name,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county,geocoded_locality_name_new,geocoded_locality_type_new,geocoded_containing_county_new
84,Michigan,Leroy Township (Ingham County),26065,Leroy Township,town,Ingham County,Leroy Township,town,Calhoun County
98,Michigan,Batavia Township (Clermont County),<NA>,MI,,,Batavia Township,town,Branch County
137,New York,Town of Clinton (Dutchess County),36027,1215,,Dutchess County,Clinton,city,Oneida County
139,New York,Town of Enfield (Tompkins County),36109,168,,Tompkins County,Enfield,city,Tompkins County
141,New York,Town of Hartland (Niagara County),36063,8942,,Niagara County,Hartland,city,Niagara County
144,New York,Town of Clifton Park (Saratoga County),36091,Town of Clifton Park,,Saratoga County,Clifton Park,town,Saratoga County
148,New York,Town of Porter (Niagara County),36063,3265,,Niagara County,Porter,town,Niagara County
149,New York,Town of Seneca (Ontario County),36069,3675,,Ontario County,Seneca County,county,Seneca County
150,New York,Town of Somerset (Niagara County),36063,Somerset Town Park,,Niagara County,Somerset,city,Niagara County
152,New York,Town of Worth (Jefferson County),36045,24625,,Jefferson County,Worth,city,Jefferson County


In [38]:
has_labeled_county = trans_ordn['raw_locality_name'].str.contains('(', regex=False)
labeled_counties = trans_ordn['raw_locality_name'].str.extract(r'.+\((.+)\)')
is_multi_county = trans_ordn['raw_locality_name'].str.contains('counties', regex=False, case=False)

In [32]:
labeled_counties.ne(trans_ordn['geocoded_containing_county'], axis=0)

,0
0,<NA>
1,<NA>
2,<NA>
3,<NA>
4,<NA>
...,...
223,False
224,False
225,<NA>
226,False


In [39]:
# where to labeled counties not match geocoded counties? (except multi-county cities)
# Looks like one typo and one error in the original data
trans_ordn.loc[
    has_labeled_county.astype("boolean")
    & ~is_multi_county
    & labeled_counties.ne(trans_ordn["geocoded_containing_county"], axis=0)
    .astype("boolean")
    .squeeze(),
    col_subset,
]

,raw_state_name,raw_locality_name,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
98,Michigan,Batavia Township (Clermont County),<NA>,MI,,
199,Texas,City of Kingsville (Kleburg County),48273,Kingsville,city,Kleberg County


In [43]:
labeled_counties.iloc[199], trans_ordn['geocoded_containing_county'].iloc[199]

(0    Kleburg County
 Name: 199, dtype: object,
 'Kleberg County')

My takeaway is that the geocoder does a better job than the Sabin Center at identifying containing counties. No parsing required.

In [44]:
ordn.loc[:, "locality"].str.contains(
        " Solar$| Wind$| Zoning Ordinance$", "", regex=True
    ).sum()

0

In [51]:
trans_ordn.iloc[139]

raw_state_name                                                         New York
raw_locality_name                             Town of Enfield (Tompkins County)
ordinance_text                The Town of Enfield imposed a moratorium on co...
state_id_fips                                                                36
county_id_fips                                                            36109
geocoded_locality_name                                                      168
geocoded_locality_type                                                         
geocoded_containing_county                                      Tompkins County
earliest_year_mentioned                                                    2017
latest_year_mentioned                                                      2019
n_years_mentioned                                                             3
Name: 139, dtype: object

In [53]:
# special case for some NY towns that get geocoded to the address of the town hall.
# The geocoding code used to simply take the first part of the address, which in these cases are street numbers.
# This is now fixed.
trans_ordn.loc[trans_ordn['geocoded_locality_name'].str.match(r'\d+'), col_subset]

,raw_state_name,raw_locality_name,county_id_fips,geocoded_locality_name,geocoded_locality_type,geocoded_containing_county
137,New York,Town of Clinton (Dutchess County),36027,1215,,Dutchess County
139,New York,Town of Enfield (Tompkins County),36109,168,,Tompkins County
141,New York,Town of Hartland (Niagara County),36063,8942,,Niagara County
148,New York,Town of Porter (Niagara County),36063,3265,,Niagara County
149,New York,Town of Seneca (Ontario County),36069,3675,,Ontario County
152,New York,Town of Worth (Jefferson County),36045,24625,,Jefferson County


In [4]:
gc = dbcp.transform.geocoding.GoogleGeocoder()

In [55]:
gc.geocode_request(name="Town of Seneca (Ontario County)", state="NY", country="US")
gc.describe()

['3675', '', 'Ontario County']

In [56]:
gc._response

{'address_components': [{'long_name': '3675',
   'short_name': '3675',
   'types': ['street_number']},
  {'long_name': 'Flint Road', 'short_name': 'Flint Rd', 'types': ['route']},
  {'long_name': 'Stanley',
   'short_name': 'Stanley',
   'types': ['locality', 'political']},
  {'long_name': 'Seneca',
   'short_name': 'Seneca',
   'types': ['administrative_area_level_3', 'political']},
  {'long_name': 'Ontario County',
   'short_name': 'Ontario County',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'New York',
   'short_name': 'NY',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'United States',
   'short_name': 'US',
   'types': ['country', 'political']},
  {'long_name': '14561', 'short_name': '14561', 'types': ['postal_code']},
  {'long_name': '9575',
   'short_name': '9575',
   'types': ['postal_code_suffix']}],
 'formatted_address': '3675 Flint Rd, Stanley, NY 14561, USA',
 'geometry': {'location': {'lat': 42.8541221, 'lng': -7